# MAC0115 2015 - Professor Routo Terada

## Solução do EP2 em python3
### Por Thales Paiva (<thalespaiva@gmail.com>)

### 0. Enunciado

Foi pedido que os alunos implementassem o método de Monte Carlo para a integração de funções de uma variável num dado intervalo $[a, b]$. Para facilitar, é feita a suposição de que, para todo $x \in [a, b]$, temos que $ 0 \leq f(x) \leq M $, para algum $M$ conhecido.

O método de Monte Carlo para estimar a integral de uma função $f$ como especificada acima é dado por:
1. Gere um conjunto $X = \{( x_1, y_1), \dots, (x_n, y_n) \}$ de $n$ pares de pontos em $ [a,b] \times [0, M] $ através de um gerador de número pseudo-aleatório.
2. Calcule $k = \#\{(x_i, y_i) \in X : y_i \leq f(x_i)\}$,o número de pontos que ficam abaixo da curva de $f$.
3. Calcule $p = \frac{k}{n} $, a proporção dos pontos abaixo da curva de $f$ no retângulo $ [a,b] \times [0, M] $.
4. Calcule $q = M(b - a)$, o valor da área do retângulo $ [a,b] \times [0, M] $.
4. A estimativa da integral será $ \int_a^b f(x) \approx pq $.

Falaremos sobre cada um destes passos nas próximas seções.

### 1. Geração do conjunto $X$

Queremos gerar $n$ pontos pseudo-aletórios sob uma distribuição uniforme no retângulo $ [a,b] \times [0, M] $. Em Python, através do módulo `random`, é muito fácil gerar um número pseudo-aleatório sob distribuição uniforme no intervalo $[r, s]$. Primeiro, importamos o módulo:

In [1]:
import random

Sempre que usamos funções com comportamento pseudo-aleatório, é necessário inicializar uma semente, que é o ponto de partida para o gerador. Há dois principais motivos opostos que fazem a inicialização de uma semente ser boa prática:
1. Quando queremos que os resultados do experimento possam ser reproduzidos. Assim, usamos uma semente pré-determinada e fazemos `random.seed(s)`, com s sendo um inteiro fixo.
2. Quando queremos dar uma semente para termos resultados diferentes cada vez que o programa for rodado. Nesse caso, podemos usar parâmetros não pré-determinados como tempo atual,  temperatura do processador ou uso da memória. Por ser comum usarmos o tempo atual, a chamada de `random.seed()` sem argumento faz exatamente isso.

In [2]:
random.seed()

In [3]:
r, s = 10, 20

In [4]:
random.uniform(r, s)

11.337181988630602

Então, para gerarmos um ponto $(x, y)$ num retângulo, digamos $[0, 4] \times [1,5]$, podemos fazer:

In [5]:
x, y = random.uniform(0, 4), random.uniform(1, 5)

In [6]:
print(x, y)

1.8321995657747756 3.500093001754777


Assim, parece natural criarmos uma função `gen_n_points_in_rectangle(n, a, b, l, m)` que tem como parâmetros $n$ e os limites do retângulo. 

In [7]:
def gen_n_points_in_rect(n, a, b, l, m):
    points = []
    
    for i in range(n):
        x, y = random.uniform(a, b), random.uniform(l, m)
        points.append((x, y))
    
    return points

E assim, para gerar 10 pontos no retângulo $[0, 4] \times [1, 5]$, podemos fazer:

In [8]:
gen_n_points_in_rect(10, 0, 4, 1, 5)

[(3.483409391480946, 3.0934108061193526),
 (2.617433862220216, 2.914886062916314),
 (0.24784972817848416, 2.4940670234238937),
 (3.2606965928545923, 4.839977954961904),
 (0.1428586173519788, 3.271337231144816),
 (1.006445179359552, 3.612593277953874),
 (1.7331022797699283, 2.394619854638689),
 (2.4067374257665173, 4.16471382872697),
 (1.095954486433393, 4.190446820999021),
 (3.220030388931838, 2.720546100987794)]

### Integração de $f$

Uma implementação possível então para a estimação da integral por Monte Carlo é a seguinte:

In [9]:
def integrate_by_monte_carlo_overmem(f, n, a, b, l, m):
    k = 0
    
    for (x, y) in gen_n_points_in_rect(n, a, b, l, m):
        if y <= f(x):
            k += 1
    
    return (k / n)*(b - a)*(m - l)
    

Porém, apesar de correta, há um problema sério de eficiência com essa função: ela guarda a lista de todos os pontos gerados na memória. Isso acontece porque primeiro a função `gen_n_points()` gera os pontos e depois estes são usados pela nossa função. Há alguns modos razoáveis de se resolver esse problema. Dois deles são:
1. Processar um ponto logo após a sua geração, descartando logo em seguida.
2. Usando o comando `yield` do python na função geradora de pontos.

Usando o primeiro método, temos uma função que é uma mistura das funções `integrate_by_monte_carlo_slow` e `gen_n_points_in_rect`.

In [10]:
def integrate_by_monte_carlo(f, n, a, b, l, m):
    k = 0
    
    for i in range(n):
        x, y = random.uniform(a, b), random.uniform(l, m)
        if y <= f(x):
            k += 1
    
    return (k / n)*(b - a)*(m - l)

Usando o segundo método, temos a função:

In [11]:
def gen_n_points_in_rect_online(n, a, b, l, m):
    
    for i in range(n):
        x, y = random.uniform(a, b), random.uniform(l, m)
        yield (x, y)
    

Chamei de online pois ela calcula o ponto (x, y) só quando ele é necessário.

In [12]:
list(gen_n_points_in_rect_online(10, 0, 4, 1, 5))

[(1.9121498718849161, 2.127951924566092),
 (2.783923592744942, 3.8582043361765077),
 (0.7670208477880163, 4.782305109320005),
 (2.7880020508687537, 2.6857860944987735),
 (2.063885503627568, 4.71662311701547),
 (2.96955433336451, 2.8985752091768457),
 (2.8368802562668747, 2.622222655345537),
 (2.755079159310944, 3.9030350215426206),
 (3.6860973137220565, 1.3401867224051984),
 (3.0762331731897183, 3.7006402502210567)]

E a função que integra seria:

In [13]:
def integrate_by_monte_carlo_online(f, n, a, b, l, m):
    k = 0
    
    for (x, y) in gen_n_points_in_rect_online(n, a, b, l, m):
        if y <= f(x):
            k += 1
    
    return (k / n)*(b - a)*(m - l)

### Eficiência das soluções

Considere a integral $ \int_1^3 x^2sin^2(x)dx \approx 4.2166 $. Primeiro importamos o módulo `math`.

In [14]:
import math

Agora, definimos a função:

In [15]:
def f1(x):
    sin_x = math.sin(x)
    return (x*x)*(sin_x*sin_x)

E definimos os limites para o método de Monte Carlo.

In [16]:
a, b, l, m = 1, 3, 0, 5

In [17]:
n0 = 1000000

Observe como as saídas são diferentes pois não reinicializamos a semente:

In [18]:
integrate_by_monte_carlo_overmem(f1, n0, a, b, l, m)

4.21244

In [19]:
integrate_by_monte_carlo_online(f1, n0, a, b, l, m)

4.22115

In [20]:
integrate_by_monte_carlo(f1, n0, a, b, l, m)

4.22054

E como ficam iguais quando a reinicializamos com valor pré-determinado, digamos 2:

In [21]:
random.seed(2); integrate_by_monte_carlo_overmem(f1, n0, a, b, l, m)

4.21432

In [22]:
random.seed(2); integrate_by_monte_carlo_online(f1, n0, a, b, l, m)

4.21432

In [23]:
random.seed(2); integrate_by_monte_carlo(f1, n0, a, b, l, m)

4.21432

E agora, sobre o desempenho delas:

In [24]:
n1 = 10000000

In [26]:
%timeit integrate_by_monte_carlo_overmem(f1, n1, a, b, l, m)

1 loops, best of 3: 17.2 s per loop


In [27]:
%timeit integrate_by_monte_carlo_online(f1, n1, a, b, l, m)

1 loops, best of 3: 16.6 s per loop


In [28]:
%timeit integrate_by_monte_carlo(f1, n1, a, b, l, m)

1 loops, best of 3: 14.9 s per loop
